In [1]:
import math
import datetime
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
#Support methods, consts, etc

def Tau_Step (tau_min):
  return tau_min * 60 / 31557600

In [2]:
class MCEngine(object):
  m_MaxP, m_MaxL = 5000, 50000;
  m_L, m_P = 0, 0

  def __init__(self, P, mu, sigma, S0):
     if (P < 0 or sigma < 0 or S0 < 0):
      raise ValueError ("Incorrect params P, sigma, S0!")
     else:
      self.m_P = P
      self.m_mu = mu
      self.m_sigma = sigma
      self.m_S0 = S0
      self.m_diff = DiffusionGBM(mu, sigma, S0)

  def Simulate(self, exp_Time, tau_min):

    t_step = Tau_Step(tau_min)
    st_step = math.sqrt(t_step)
    sim_time = exp_Time / 365.25
    L = int(sim_time / t_step)


    if (self.m_P * self.m_L > self.m_MaxP * self.m_MaxL):
      raise ValueError ("Incorrect params L, P!")
    else:
      self.m_L = L
      self.path = np.zeros((self.m_P,self.m_L))
      print("L = ", L)
  
    SumELn, SumE2Ln, Emu, Esigma2 = 0, 0, 0, 0
    np.random.seed()
    for s in range(self.m_P):
      self.path[s][0] = self.m_S0

    ind = int(self.m_P/2)
    print("P/2 = ", ind)

    print("tau_step = ", t_step)
    print("st_step = ", st_step)

    StEnd = np.zeros(self.m_P)

    for p in range(ind):
      St0 = self.m_S0
      St1 = St0
      Mu0 = 0
      Mu1 = 0
      for l in range(1, self.m_L):
        
        rnd = np.random.normal(0)
        Mu0 = self.m_diff.Mu(St0)
        Mu1 = self.m_diff.Mu(St1)
        Sigma0 = self.m_diff.Sigma(St0)
        Sigma1 = self.m_diff.Sigma(St1)

        #print("Mu0 = ", Mu0, ", Mu1 = ", Mu1)
        #print("Random = ", rnd, "Sig0 = ", Sigma0, ", Sig1 = ", Sigma1)

        St0 = self.path[p][l-1] + Mu0 * t_step + Sigma0 * rnd * st_step
        St1 = self.path[ind + p][l-1] + Mu1 * t_step - Sigma1 * rnd * st_step
        self.path[p][l] = St0        
        self.path[ind + p][l] = St1
      
      StEnd[p] = St0
      StEnd[ind + p] = St1

        #print("Random = ", rnd, ", St0 = ", St0, ", St1 = ", St1)

    EST = 0
    EST2 = 0
    for p in range(self.m_P):
      RT = math.log(StEnd[p]/self.m_S0)
      EST += RT
      EST2 += RT * RT 

    EST /= self.m_P
    VarSt = (EST2 - self.m_P * EST * EST) / (self.m_MaxP - 1)
    print("Var = ", VarSt)
    Esigma2 = VarSt / sim_time
    Emu = (EST + VarSt / 2) / sim_time
    
    print("Mu =", self.m_mu, ", sigma2 = ", self.m_sigma * self.m_sigma)
    print("E[mu] =", Emu, ", E[sigma2] = ", Esigma2)

  def ret_path(self, np, nl):
    return self.path[np][nl]

In [3]:
class DiffusionGBM(object):

  #by default
  m_mu, m_sigma, m_S0 = 0.15, 0.25, 100.0

  def __init__(self, mu, sigma, S0):
    if (sigma < 0 or S0 < 0):
      raise ValueError ("Incorrect params!")
    else:
      self.m_mu, self.m_sigma, self.m_S0 = mu, sigma, S0
      print("GBM is initiated:", mu, sigma, S0)

  def Mu(self, st):
    return st * self.m_mu if st * self.m_mu > 0 else 0

  def Sigma(self, st):
    return st * self.m_sigma if st * self.m_sigma > 0 else 0

  def GetStartPrice(self):
    return self.m_S0

In [4]:
#MCEngine(self, P, mu, sigma, S0)
#Simulate(self, exp_Time, tau_min)
P = 10000
mu = 0.25
sigma = 0.25
S0 = 100
tau_mins = 60 #minutes
exp_time = 30 #days

mc = MCEngine(P, mu, sigma, S0)
mc.Simulate(exp_time, tau_mins)

GBM is initiated: 0.25 0.25 100
L =  720
P/2 =  5000
tau_step =  0.00011407711613050422
st_step =  0.010680688935200024
Var =  0.010510953406780205
Mu = 0.25 , sigma2 =  0.0625
E[mu] = 0.2824154438008828 , E[sigma2] =  0.12797085772754901
